In [1]:
import numpy as np
import pandas as pd

## Bring in the voteview data, and filter to the 118th congress

In [2]:
ideology = pd.read_csv('https://voteview.com/static/data/out/members/HS118_members.csv')

In [3]:
ideology['chamber'].value_counts()

chamber
House     441
Senate    100
Name: count, dtype: int64

In [8]:
ideology[['bioname', 'chamber', 'party_code', 
          'state_abbrev', 'district_code', 'nominate_dim1']]\
    .sort_values('nominate_dim1', ascending=True)




,bioname,chamber,party_code,state_abbrev,district_code,nominate_dim1
382,"GARCIA, Sylvia",House,100,TX,29,-0.807
482,"WARREN, Elizabeth",Senate,100,MA,0,-0.752
71,"LEE, Barbara",House,100,CA,12,-0.677
143,"CASTEN, Sean",House,100,IL,6,-0.672
66,"WATERS, Maxine",House,100,CA,43,-0.656
...,...,...,...,...,...,...
527,"LEE, Mike",Senate,200,UT,0,0.891
474,"PAUL, Rand",Senate,200,KY,0,0.891
320,"BRECHEEN, Josh",House,200,OK,2,0.926
441,"TUBERVILLE, Thomas Hawley (Tommy)",Senate,200,AL,0,0.936


In [10]:
replace_map = {100: 'Democrat',
              200: 'Republican',
              328: 'Independent'}
ideology['party_code'] = ideology['party_code'].replace(replace_map)
ideology['party_code'].value_counts()

party_code
Republican     274
Democrat       264
Independent      3
Name: count, dtype: int64

In [13]:
ideology.query("party_code == 'Independent'")[['bioname', 'chamber', 
                                               'state_abbrev', 'nominate_dim1']]

,bioname,chamber,state_abbrev,nominate_dim1


In [16]:
ideology[['chamber', 'icpsr', 'bioname']]

,chamber,icpsr,bioname
0,House,20301,"ROGERS, Mike Dennis"
1,House,21102,"SEWELL, Terri"
2,House,21500,"PALMER, Gary James"
3,House,22108,"CARL, Jerry L."
4,House,22140,"MOORE, Barry"
...,...,...,...
536,Senate,40915,"MANCHIN, Joe, III"
537,Senate,29940,"BALDWIN, Tammy"
538,Senate,41111,"JOHNSON, Ron"
539,Senate,20953,"LUMMIS, Cynthia M."


In [20]:
ideology.query("state_abbrev == 'VA' & chamber == 'House'")[['district_code', 'party_code', 'bioname']]

,district_code,party_code,bioname
409,1,Republican,"WITTMAN, Robert J."
410,11,Democrat,"CONNOLLY, Gerald E. (Gerry)"
411,9,Republican,"GRIFFITH, H. Morgan"
412,8,Democrat,"BEYER, Donald Sternoff Jr."
413,6,Republican,"CLINE, Benjamin"
414,7,Democrat,"SPANBERGER, Abigail"
415,10,Democrat,"WEXTON, Jennifer"
416,5,Republican,"GOOD, Bob"
417,2,Republican,"KIGGANS, Jennifer"
418,4,Democrat,"MCCLELLAN, Jennifer"


In [22]:
url = 'https://voteview.com/static/data/out/votes/HS118_votes.csv'
votes = pd.read_csv(url)
votes

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,118,House,1,14854,6,100.0
1,118,House,1,14863,6,98.0
2,118,House,1,14873,1,100.0
3,118,House,1,15029,1,100.0
4,118,House,1,15433,1,100.0
...,...,...,...,...,...,...
248232,118,Senate,247,42303,6,91.4
248233,118,Senate,247,42304,6,63.7
248234,118,Senate,247,49308,1,100.0
248235,118,Senate,247,49703,1,100.0


In [24]:
# First do the merge in a way that checks for problems
ideology_votes = pd.merge(votes, ideology,
                         on = 'icpsr',
                         how = 'outer',
                         validate = 'many_to_one',
                         indicator = 'matched')

In [25]:
ideology_votes['matched'].value_counts()

matched
both          248237
left_only          0
right_only         0
Name: count, dtype: int64

In [26]:
# Second, if the checks all pass, do the merge in a way that is more memory efficient
ideology_votes = pd.merge(votes, ideology,
                         on = 'icpsr',
                         how = 'inner')
ideology_votes

,congress_x,chamber_x,rollnumber,icpsr,cast_code,prob,congress_y,chamber_y,state_icpsr,district_code,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,118,House,1,14854,6,100.0,118,House,51,5,...,NaN,0.336,0.135,-47.23858,0.90240,460,10,NaN,0.365,0.754
1,118,House,2,14854,6,100.0,118,House,51,5,...,NaN,0.336,0.135,-47.23858,0.90240,460,10,NaN,0.365,0.754
2,118,House,3,14854,6,100.0,118,House,51,5,...,NaN,0.336,0.135,-47.23858,0.90240,460,10,NaN,0.365,0.754
3,118,House,4,14854,6,100.0,118,House,51,5,...,NaN,0.336,0.135,-47.23858,0.90240,460,10,NaN,0.365,0.754
4,118,House,5,14854,6,100.0,118,House,51,5,...,NaN,0.336,0.135,-47.23858,0.90240,460,10,NaN,0.365,0.754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248232,118,Senate,243,91300,6,99.8,118,Senate,61,0,...,NaN,-0.303,0.532,-18.40028,0.92375,232,7,NaN,-0.303,0.532
248233,118,Senate,244,91300,1,100.0,118,Senate,61,0,...,NaN,-0.303,0.532,-18.40028,0.92375,232,7,NaN,-0.303,0.532
248234,118,Senate,245,91300,1,99.8,118,Senate,61,0,...,NaN,-0.303,0.532,-18.40028,0.92375,232,7,NaN,-0.303,0.532
248235,118,Senate,246,91300,1,100.0,118,Senate,61,0,...,NaN,-0.303,0.532,-18.40028,0.92375,232,7,NaN,-0.303,0.532
